In [1]:
# coding=utf-8
import sys

sys.path.append('../')

from util import load_json
from data_loader.song_info_loader import SongMetaInfoLoader

from tqdm.auto import tqdm

In [2]:
TRAIN_DATA_PATH = '../res/orig/train.json'
META_DATA_PATH = '../res/song_meta.json'

In [3]:
train_data = load_json(TRAIN_DATA_PATH)
meta_data = SongMetaInfoLoader()
meta_data.make_info_map(playlist_data=train_data, meta_data=load_json(META_DATA_PATH))

make meta info of songs!



done!


In [4]:
aid_album_id_dict = dict(enumerate(sorted(set(meta_data.song_album_id_map.values()))))
album_id_aid_dict = dict(map(lambda kv: (kv[1], kv[1]), aid_album_id_dict.items()))

In [5]:
def playlist_to_aids(playlist):
    aid_list = []
    for plays in tqdm(playlist):
        aids = list(map(lambda s: str(meta_data.song_album_id_map[s]), plays['songs']))
        aid_list.append(aids)
        
    return aid_list

In [6]:
aid_seqs = playlist_to_aids(playlist=train_data)

In [52]:
sum(map(lambda e: len(e), aid_seqs))

4239978

In [8]:
from collections import Counter

In [15]:
aid_counter = Counter([aid for sub_seq in aid_seqs for aid in sub_seq]).most_common()

In [62]:
total_freq = sum(map(lambda e: len(e), aid_seqs))
acc = 0
for cnt_idx in range(0, len(aid_counter)):
    aid, cnt = aid_counter[cnt_idx]
    acc += cnt
    if cnt_idx % 1000 == 0:
        print(f"{cnt_idx}, {aid}, {cnt}, {acc * 100 / total_freq:.3f}")
#     break

0, 2161126, 4071, 0.096
1000, 311295, 544, 21.738
2000, 10263118, 352, 31.953
3000, 10252753, 255, 38.985
4000, 2125020, 199, 44.305
5000, 10094403, 162, 48.535
6000, 10292688, 135, 52.012
7000, 2179131, 116, 54.959
8000, 10377359, 102, 57.534
9000, 2681085, 90, 59.796
10000, 2138528, 81, 61.806
11000, 10037829, 73, 63.615
12000, 2317098, 66, 65.243
13000, 2283226, 60, 66.722
14000, 10034886, 55, 68.078
15000, 2704355, 51, 69.325
16000, 10184702, 47, 70.478
17000, 2039758, 44, 71.550
18000, 348865, 41, 72.546
19000, 10137358, 38, 73.482
20000, 307320, 36, 74.358
21000, 2333097, 34, 75.181
22000, 2687645, 32, 75.956
23000, 15883, 30, 76.690
24000, 2304486, 29, 77.384
25000, 10138006, 27, 78.038
26000, 2168047, 26, 78.661
27000, 2304302, 25, 79.255
28000, 2193730, 23, 79.820
29000, 317519, 22, 80.360
30000, 397298, 21, 80.877
31000, 17090, 20, 81.372
32000, 2639445, 20, 81.844
33000, 1128955, 19, 82.296
34000, 10196411, 18, 82.731
35000, 10098925, 17, 83.152
36000, 2019246, 17, 83.553
37

In [64]:
import tensorflow as tf

In [65]:
tf.keras.preprocessing.text.Tokenizer?

In [66]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', num_words=60000)

In [67]:
tokenizer.fit_on_texts(aid_seqs)

In [80]:
tokenizer.texts_to_sequences(['2302415 2326467 714453 862952 860568 4234823'])

[[255, 363, 6269, 2536, 1332]]

In [82]:
from tensorflow import keras
from tensorflow.keras import layers

In [83]:
class MultiHeadSelfAttention(layers.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = layers.Dense(embed_dim)
        self.key_dense = layers.Dense(embed_dim)
        self.value_dense = layers.Dense(embed_dim)
        self.combine_heads = layers.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output
    
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)
    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=emded_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions
    
class TokenEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, emded_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=emded_dim)

    def call(self, x):
        x = self.token_emb(x)
        return x